In [2]:
let user = {
    name: 'John',
    age: 30,
    
    toString() {
        return `{name: "${this.name}", age: ${this.age}}`;
    }
};

In [3]:
console.log(user);

{ name: 'John', age: 30, toString: [Function: toString] }


### JSON.stringify

JavaScript provides methods:

* `JSON.stringify` to convert objects into JSON.
* `JSON.parse` to convert JSON back into an object.

In [4]:
let student = {
    name: "John",
    age: 30,
    isAdmin: false,
    enrolledCourses: [
        'Operating System',
        'Data Structures',
        'Database Management System',
        'Distributed System'
    ],
    spouse: null,
};

In [5]:
let jsonStudent = JSON.stringify(student);
jsonStudent;

'{"name":"John","age":30,"isAdmin":false,"enrolledCourses":["Operating System","Data Structures","Database Management System","Distributed System"],"spouse":null}'

In [6]:
typeof(jsonStudent);

'string'

JSON supports following data types:

* Objects `{ ... }`
* Arrays `[ ... ]`
* Primitives:
    * strings,
    * numbers,
    * boolean values `true/false`,
    * null.

In [7]:
JSON.stringify(1);

'1'

In [8]:
JSON.stringify('hello');

'"hello"'

In [9]:
JSON.stringify(true);

'true'

In [10]:
JSON.stringify([1, 2, 3]);

'[1,2,3]'

In [11]:
JSON.stringify(null);

'null'

JSON is data-only language-independent specification, so some JavaScript-specific object properties are skipped by `JSON.stringify`.

* Function properties (methods).
* Symbolic keys and values.
* Properties that store `undefined`.

In [12]:
user = {
    sayHi() {
        console.log('Hello');
    },
    [Symbol('id')]: 123,
    something: undefined,
};

user;

{ sayHi: [Function: sayHi], something: undefined, [Symbol(id)]: 123 }

In [13]:
JSON.stringify(user);

'{}'

In [14]:
// nested objects are supported and converted

let meetup = {
    title: 'Conference',
    room: {
        number: 23,
        participants: [
            'John',
            'Anna'
        ],
    },
};

JSON.stringify(meetup);

'{"title":"Conference","room":{"number":23,"participants":["John","Anna"]}}'

> The important limitation: there must be no circular references.

In [15]:
let room = {
    number: 23
};

meetup = {
    title: 'Conference',
    participants: [
            'John',
            'Anna'
    ],
};

meetup.place = room; // meetup references room
room.occupiedBy = meetup; // room references meetup

JSON.stringify(meetup);

TypeError: Converting circular structure to JSON
    --> starting at object with constructor 'Object'
    |     property 'place' -> object with constructor 'Object'
    --- property 'occupiedBy' closes the circle

Here, the conversion fails, because of circular reference: `room.occupiedBy` references `meetup`, and `meetup.place` references `room`:


![circular-references](https://i.ibb.co/mh0vmFK/image.png)

> Full Syntax: `JSON.stringify(value[, replacer, space])`

In [16]:
JSON.stringify(room);

TypeError: Converting circular structure to JSON
    --> starting at object with constructor 'Object'
    |     property 'occupiedBy' -> object with constructor 'Object'
    --- property 'place' closes the circle

### Excluding and transforming: replacer

In [17]:
room = {
    number: 23,
};

meetup = {
    title: 'Conference',
    participants: [
        {name: 'John'},
        {name: 'Alice'},
    ],
    place: room,
};

room.occupiedBy = meetup; // room references meetup

<ref *1> {
  title: 'Conference',
  participants: [ { name: 'John' }, { name: 'Alice' } ],
  place: { number: 23, occupiedBy: [Circular *1] }
}

In [18]:
JSON.stringify(room, ['number']);

'{"number":23}'

In [19]:
JSON.stringify(meetup, [
    'title',
    'participants'
]);

// expected output: {"title":"Conference","participants":[{},{}]}

'{"title":"Conference","participants":[{},{}]}'

In [20]:
JSON.stringify(meetup, [
    'title',
    'participants',
    'place',
    'name',
    'number'
]); 

// Expected output: {"title":"Conference","participants":[{"name":"John"},{"name":"Alice"}],"place":{"number":23}}

'{"title":"Conference","participants":[{"name":"John"},{"name":"Alice"}],"place":{"number":23}}'

In [21]:
meetup;

<ref *1> {
  title: 'Conference',
  participants: [ { name: 'John' }, { name: 'Alice' } ],
  place: { number: 23, occupiedBy: [Circular *1] }
}

In [22]:
room;

<ref *1> {
  number: 23,
  occupiedBy: {
    title: 'Conference',
    participants: [ [Object], [Object] ],
    place: [Circular *1]
  }
}

we can use a function instead of an array as the `replacer`.

The function will be called for every `(key, value)` pair and should return the “replaced” value, which will be used instead of the original one. Or `undefined` if the value is to be skipped.

In [23]:
room = {
  number: 23
};

meetup = {
  title: "Conference",
  participants: [{name: "John"}, {name: "Alice"}],
  place: room // meetup references room
};

room.occupiedBy = meetup; // room references meetup

JSON.stringify(meetup, (key, value) => {
    console.log(`${key}: ${value}`);
    return (key == 'occupiedBy') ? undefined: value;
});

: [object Object]
title: Conference
participants: [object Object],[object Object]
0: [object Object]
name: John
1: [object Object]
name: Alice
place: [object Object]
number: 23
occupiedBy: [object Object]


'{"title":"Conference","participants":[{"name":"John"},{"name":"Alice"}],"place":{"number":23}}'

Please note that `replacer` function gets every `key/value` pair including nested objects and array items. It is applied recursively. The value of `this` inside `replacer` is the object that contains the current property.

The first call is special. It is made using a special “wrapper object”: `{"": meetup}`. In other words, the first `(key, value)` pair has an empty key, and the value is the target object as a whole. That’s why the first line is `":[object Object]"` in the example above.

### Formatting: space

In [24]:
user = {
   name: "John",
  age: 25,
  roles: {
    isAdmin: false,
    isEditor: true
  } 
};

JSON.stringify(user, null, 2);

'{\n' +
  '  "name": "John",\n' +
  '  "age": 25,\n' +
  '  "roles": {\n' +
  '    "isAdmin": false,\n' +
  '    "isEditor": true\n' +
  '  }\n' +
  '}'

In [25]:
JSON.stringify(user, null, 4);

'{\n' +
  '    "name": "John",\n' +
  '    "age": 25,\n' +
  '    "roles": {\n' +
  '        "isAdmin": false,\n' +
  '        "isEditor": true\n' +
  '    }\n' +
  '}'

In [26]:
JSON.stringify(user, null, space=2);

'{\n' +
  '  "name": "John",\n' +
  '  "age": 25,\n' +
  '  "roles": {\n' +
  '    "isAdmin": false,\n' +
  '    "isEditor": true\n' +
  '  }\n' +
  '}'

In [27]:
JSON.stringify(user, space=2);

'{"name":"John","age":25,"roles":{"isAdmin":false,"isEditor":true}}'

In [28]:
JSON.stringify(value=user, replacer=null, space=2);

'{\n' +
  '  "name": "John",\n' +
  '  "age": 25,\n' +
  '  "roles": {\n' +
  '    "isAdmin": false,\n' +
  '    "isEditor": true\n' +
  '  }\n' +
  '}'

The third argument can also be a string. In this case, the string is used for indentation instead of a number of spaces.

In [29]:
JSON.stringify(value=user, replacer=null, space="2");

'{\n' +
  '2"name": "John",\n' +
  '2"age": 25,\n' +
  '2"roles": {\n' +
  '22"isAdmin": false,\n' +
  '22"isEditor": true\n' +
  '2}\n' +
  '}'

### Custom "toJSON" Implementation

In [30]:
room = {
    number: 23,
};

meetup = {
    title: 'Conference',
    date: new Date().toLocaleDateString(), // toJSON() method is built in date object
    room,
};

JSON.stringify(meetup);

'{"title":"Conference","date":"6/2/2023","room":{"number":23}}'

In [31]:
// Custom toJSON

room = {
    number: 23,
    toJSON() {
        return 'Hello';
    },
};

JSON.stringify(room);

'"Hello"'

In [32]:
room = {
    number: 23,
    toJSON() {
        return this.number;
    },
};

meetup = {
    title: 'Conference',
    room,
};

JSON.stringify(room);

'23'

In [33]:
JSON.stringify(meetup);

'{"title":"Conference","room":23}'

### JSON.parse

> Full Syntax: `JSON.parse(str, [reviver]);`

In [34]:
let numbers = "[0, 1, 2, 3]";
JSON.parse(str=numbers);

[ 0, 1, 2, 3 ]

In [35]:
let userData = '{ "name": "John", "age": 35, "isAdmin": false, "friends": [0,1,2,3] }';
JSON.parse(str=userData);

{ name: 'John', age: 35, isAdmin: false, friends: [ 0, 1, 2, 3 ] }

In [36]:
str = '{"title":"Conference","date":"2017-11-30T12:00:00.000Z"}';
meetup = JSON.parse(str);
meetup.date.getDate();

TypeError: meetup.date.getDate is not a function

The value of `meetup.date` is a string, not a `Date` object. How could `JSON.parse` know that it should transform that string into a `Date`?

Let’s pass to `JSON.parse` the `reviving` function as the second argument, that returns all values “as is”, but date will become a `Date`:

In [37]:
str = '{"title":"Conference","date":"2017-11-30T12:00:00.000Z"}';
meetup = JSON.parse(
    str=str, reviver=(key, value) => {
        if (key === 'date') {
            return new Date(value);
        } else {
            return value;
        }
    }
);

meetup;

{ title: 'Conference', date: 2017-11-30T12:00:00.000Z }

In [38]:
meetup.date.getDate();

30

In [39]:
meetup.date.getFullYear();

2017

### Summary

* JSON is a data format that has its own independent standard and libraries for most programming languages.
* JSON supports plain objects, arrays, strings, numbers, booleans, and null.
* JavaScript provides methods `JSON.stringify` to serialize into JSON and `JSON.parse` to read from JSON.
* Both methods support transformer functions for smart reading/writing.
* If an object has `toJSON`, then it is called by `JSON.stringify`.

### Tasks

#### Task 1

Turn the user into JSON and then read it back into another variable.

In [40]:
user = {
  name: "John Smith",
  age: 35
};

{ name: 'John Smith', age: 35 }

In [41]:
let serializedData = JSON.stringify(value=user);
serializedData;

'{"name":"John Smith","age":35}'

In [42]:
let parsedData = JSON.parse(str=serializedData);
parsedData;

{ name: 'John Smith', age: 35 }

#### Task 2

In simple cases of circular references, we can exclude an offending property from serialization by its name.

But sometimes we can’t just use the name, as it may be used both in circular references and normal properties. So we can check the property by its value.

Write `replacer` function to stringify everything, but remove properties that reference `meetup`:

In [43]:
room = {
    number: 25,
};

meetup = {
    title: "Conference",
    place: room,
    occupiedBy: [
        {name: 'John'},
        {name: 'Alice'}
    ],
};

{
  title: 'Conference',
  place: { number: 25 },
  occupiedBy: [ { name: 'John' }, { name: 'Alice' } ]
}

In [44]:
// circular references
room.occupiedBy = meetup;
meetup.self = meetup;

<ref *1> {
  title: 'Conference',
  place: { number: 25, occupiedBy: [Circular *1] },
  occupiedBy: [ { name: 'John' }, { name: 'Alice' } ],
  self: [Circular *1]
}

In [45]:
Object.keys(meetup);

[ 'title', 'place', 'occupiedBy', 'self' ]

In [46]:
Object.values(meetup);

[
  'Conference',
  <ref *1> {
    number: 25,
    occupiedBy: <ref *2> {
      title: 'Conference',
      place: [Circular *1],
      occupiedBy: [Array],
      self: [Circular *2]
    }
  },
  [ { name: 'John' }, { name: 'Alice' } ],
  <ref *2> {
    title: 'Conference',
    place: <ref *1> { number: 25, occupiedBy: [Circular *2] },
    occupiedBy: [ [Object], [Object] ],
    self: [Circular *2]
  }
]

In [47]:
Object.entries(meetup);

[
  [ 'title', 'Conference' ],
  [ 'place', { number: 25, occupiedBy: [Object] } ],
  [ 'occupiedBy', [ [Object], [Object] ] ],
  [
    'self',
    <ref *1> {
      title: 'Conference',
      place: [Object],
      occupiedBy: [Array],
      self: [Circular *1]
    }
  ]
]

In [48]:
serializedData = JSON.stringify(value=meetup, replacer=(key, value) => {
    return (key != "" && value===meetup) ? undefined: value;
});

serializedData;

'{"title":"Conference","place":{"number":25},"occupiedBy":[{"name":"John"},{"name":"Alice"}]}'